In [1]:
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [2]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
log_period_ms = 50

## testing camera

In [3]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Compute
    cv2.imshow('frame', frame)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()
del cap

## helper functions

In [4]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)
        
def ascend_and_hover(cf):
    print('ascending')
            
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)

    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    
def hover_and_descend(cf):
    print('descending')
    
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
        
def position_callback(timestamp, data, log_conf):
    global x_cur, y_cur, z_cur
    x_cur = data['kalman.stateX']
    y_cur = data['kalman.stateY']
    z_cur = data['kalman.stateZ']

# loop

In [7]:
x_cur = 0.0
y_cur = 0.0
z_cur = 0.0

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('FAILED! FAILED! FAILED!')
    # TODO: just exit out
else:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf
        
        # set up logging
        log_conf = LogConfig(name='Position', period_in_ms=log_period_ms)
        log_conf.add_variable('kalman.stateX', 'float')
        log_conf.add_variable('kalman.stateY', 'float')
        log_conf.add_variable('kalman.stateZ', 'float')

        scf.cf.log.add_config(log_conf)
        log_conf.data_received_cb.add_callback(position_callback)
        log_conf.start()
        
        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False
        
        # start to hover
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        # detect objects in each frame of the video
#         while cap.isOpened() and not exit_loop:
            
#             # Try to read image
#             ret, frame = cap.read()
#             if ret:
# #                 print("ret")
#                 image = frame
#                 image_height, image_width, _ = image.shape

#                 # create blob from image
#                 blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
#                                              swapRB=True)

#                 # forward propagate image
#                 model.setInput(blob)
#                 detections = model.forward()

#                 # select detections that match selected class label
#                 matching_detections = [d for d in detections[0, 0] if d[1] == tracking_label]
# #                 print("matching_detections", matching_detections)

#                 # select confident detections
#                 confident_detections = [d for d in matching_detections if d[2] > confidence]

#                 # get detection closest to center of field of view and draw it
#                 det = closest_detection(confident_detections) # This relies on the function you wrote above
                
#                 if det is not None:
#                     # get the class id
#                     class_id = det[1]
#                     # map the class id to the class 
#                     class_name = class_names[int(class_id)-1]
#                     color = COLORS[int(class_id)]
#                     # get the bounding box coordinates
#                     box_x = det[3] * image_width
#                     box_y = det[4] * image_height
#                     # get the bounding box width and height
#                     box_width = det[5] * image_width
#                     box_height = det[6] * image_height
#                     # draw a rectangle around each detected object
#                     cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
#                     # put the class name text on the detected object
#                     cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

#                 # If nothing is detected, hover
#                 if det is None:
#                     print('no detection...hovering')
#                     hover(cf)

#                 # otherwise  move towards target
#                 else:
#                     print('detection...tracking')
#                     print('conf', det[2])
#                     _, _, _, box_x, box_y, box_width, box_height = det
#                     box_x, box_y = detection_center(det)
#                     exit_loop, x_cur, y_cur = controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur)

#                 # Check image
#                 cv2.imshow('image', image)
#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break
                    
#             else:
#                 print('no image!!')
                
#         cap.release()
        
        # descend
        hover_and_descend(cf)
        
#         cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Error no LogEntry to handle id=1
Error no LogEntry to handle id=1


Crazyflies found:
radio://0/4/2M
radio://0/4/2M


OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!


Initializing PID Controller
Waiting for estimator to find position...
999.9999889124201 999.9999889119617 999.9997975926381
999.9999897598391 999.9999897597081 999.9998105558334
999.9999897598391 999.9999897597081 999.9998105558334
999.9999897598391 999.9999897597081 999.9998105558334
999.9999897598391 999.9999897597081 999.9998116870556
999.9999897598391 999.9999897597081 999.9998116870556
999.9999897598391 999.9999897597081 999.9998116870556
999.9999897598391 999.9999897597081 999.9998116870556
999.9999897897515 999.9999897898442 999.9998116870556
9.503310138825327e-07 9.510677045909688e-07 2.3351472918875515e-05
ascending
descending
